In [1]:
!pip install ortools
#!pip install dtale

     |████████████████████████████████| 32.7MB 106kB/s 


In [2]:
import pandas as pd
import numpy as np
import collections
from ortools.sat.python import cp_model
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 176 (delta 75), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (176/176), 2.15 MiB | 5.53 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/cloned-repo


In [3]:
#Importing Data
df=pd.read_excel('orders1.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')
 
# Formating Data
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
del df['Dispatch Date']
del df['SO Date']
del df['Description']
del df['Customer']
df['Order Qty'] = df[df['Order Qty']!=0]['Order Qty']
df['Order Qty'] = df['Order Qty']*1000
df=df[df['Key']!='17023287gd39000800iX']
 
 
# Creating Parameter Data
jobs =df['Key']
op =[1,2,3,4]  # Ignore as of now
mc = [1,2,3,4,5,6]
#horizon = max(df["due"])+900
due = df.set_index('Key')
dur=df[['Key','FG Code','Order Qty']]
dur=dur.merge(pa[["FG Code","Line","Output"]], on="FG Code")
dur['Output']=dur['Output']*100
dur['duration']=dur['Order Qty']/dur['Output']
dur['duration'] = dur['duration'].astype('float64').round(decimals = 2)
dur.set_index(['Key','Line'],inplace=True)
del dur['FG Code']
dur['Output'].replace(0,1,inplace=True)
df['Order Qty']=df['Order Qty'].fillna(0).astype('int')
dur['duration']=dur['duration'].fillna(0).astype('int')
dur['Output']=dur['Output'].fillna(0).astype('int')
horizon = sum(dur['duration']*10)
 
#Setup
setup = pd.DataFrame(columns=('Key1','Key2','setup'))
for i in df['Key']:
  for k in df['Key']:
    if df[df['Key']==i]["FG Code"].values[0]==df[df['Key']==k]["FG Code"].values[0]:
      f = {'Key1':[i],"Key2":[k],"setup":[0]}
    else:
      f = {'Key1':[i],"Key2":[k],"setup":[4]}
    f = pd.DataFrame(data=f)
    setup = pd.concat([setup,f])
setup.set_index(['Key1','Key2'],inplace=True)

In [7]:
model = cp_model.CpModel()
task_type = collections.namedtuple('task_type', 'start end dur run1')
mtj = collections.defaultdict(list)
all_tasks = {}
 
for j in jobs:
  for m in mc:
      suffix = '_%s_%i' % (j,m)
      start = model.NewIntVar(0, horizon, "start"+suffix)
      end = model.NewIntVar(0, horizon, "end"+suffix)
      run1 = model.NewIntVar(0, dur['duration'].loc[(j,m)].item(), "run"+suffix)
      duration = model.NewIntervalVar(start, run1, end, "duration"+suffix)
      all_tasks[j, m] = task_type(start=start, end=end, dur= duration, run1=run1)
      mtj[m].append(duration)
 
for m in mc:
  model.AddNoOverlap(mtj[m])
'''  
for i in jobs:
  for m in mc:
    model.Add(all_tasks[i, m].run1==all_tasks[i, m].end-all_tasks[i, m].start)
'''
 
#Sequence
 
b={}
for j in jobs:
  for m in mc:
    for i in jobs:
      b[j,m,i] = model.NewBoolVar('selection')
      #model.Add(all_tasks[j, m,i].run1>0).OnlyEnforceIf(b[j,m,i])
      #model.Add(all_tasks[j, m,i].run1==0).OnlyEnforceIf(b[j,m,i].Not())
      #model.Add(all_tasks[i, m].end==0).OnlyEnforceIf(b[i,m].Not())
 
#[model.Add(sum(b[j,m,i] for i in jobs) == 1) for m in mc for j in jobs]
 
com = {}
for j in jobs:
  tt= 0
  for m in mc:
    comp = all_tasks[j,m].run1*dur["Output"].loc[(j,m)]
    tt = tt + comp
  com[j] = tt
  model.Add(com[j]*10 >= df[df["Key"]==j]['Order Qty'].values[0])
 
 
a={}
for i in jobs:
  for m in mc:
    if all_tasks[i,m].run1>0:
      for k in jobs:
        if k != i:
          a[i,k,m] = model.NewBoolVar('previous')
     
for j in jobs:
  for m in mc:
    for i in jobs:
      if j != i:
        model.AddBoolOr([a[j,i,m],a[i,j,m]])
        model.Add(all_tasks[i, m].start >= all_tasks[j, m].end + setup.loc[(j,i)].values[0]).OnlyEnforceIf(a[j,i,m]).OnlyEnforceIf(b[j,m,i])
        model.Add(all_tasks[j, m].start >= all_tasks[i, m].end + setup.loc[(j,i)].values[0]).OnlyEnforceIf(b[j,m,i]).OnlyEnforceIf(a[i,j,m])
 
'''
com1 = {}
for j in df['Key']:
  tt= 0
  for m in mc:
    comp1 = (all_tasks[j,m].end - all_tasks[j,m].start)*dur["Output"].loc[(j,m)]
    tt = tt + comp1
  com1[j] = tt
  model.Add(com1[j]*10 <= df[df["Key"]==j]['Order Qty'].values[0]+500)
'''
 
ck = model.NewIntVar(-horizon, horizon, 'dummy')
model.AddMaxEquality(ck,[all_tasks[j,m].end for j in jobs for m in mc])
 
model.Minimize(ck)
solver = cp_model.CpSolver()
solver.Solve(model)
print(solver.StatusName(),solver.ObjectiveValue())

FEASIBLE 102.0


In [6]:
ff=pd.DataFrame()
for m in mc:
  for j in jobs:
    if solver.Value(all_tasks[j,m].run1)>0:
      kk={'Key':j,'line':m,'start':solver.Value(all_tasks[j,m].start),
          'run':solver.Value(all_tasks[j,m].run1),'end':solver.Value(all_tasks[j,m].end)}
      ff=ff.append(kk,ignore_index=True)
 
#Exploring Data
!pip install dtale
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
show(ff,ignore_duplicate=True)

     |████████████████████████████████| 7.6MB 6.2MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 645kB 42.4MB/s 
     |████████████████████████████████| 1.8MB 34.7MB/s 
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 194kB 34.0MB/s 
     |████████████████████████████████| 358kB 45.7MB/s 
     |████████████████████████████████| 952kB 41.6MB/s 
     |████████████████████████████████| 3.4MB 36.7MB/s 
     |████████████████████████████████| 194kB 46.3MB/s 
     |████████████████████████████████| 1.8MB 33.4MB/s 
  Created wheel for dash-colorscales: filename=dash_colorscales-0.0.4-cp36-none-any.whl size=62599 sha256=04adfbe666dfff426b1020512ff4be2dff01f7c71bdab7f408e7a7396d8520bd
  Stored in directory: /root/.cache/pip/wheels/0e/4a/1c/3490cfe6a59299b06917af6626a969c8fb7caead39b2d74cf2
  Created wheel for Flask-Compress: filename=Flask_Compress-1.5.0-cp36-none-any.whl size=5272 sha256=7501b1a099

https://ypk46sxxn1-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

**Indices**  
j - job  
m - machine  

**Decision Variable**  
$s_{j,m}$ - start time of job j at machine m  
$e_{j,m}$ - end time of job j at machine m  
$d_{j,m}$ - duration of job j at machine m  

**Parameters**  
H - Horizon  
$d_j$ - delivery date of job j  
$q_j$ - order qty of job j  
$o_{j,m}$ - output per hour of job j at machine m  

**Objective**  
$ \sum_{j} d_j - max(e_{j,m})  \qquad \forall m \in mc$  

**Constraints**  
$ \sum_m e_{j,m} < s_{i,m} or s_{j,m} > e_{i,m} \qquad \forall j,i \in jobs$  
$ \sum_j o_{j,m} \times d_{j,m} >= q_j \qquad \forall m \in ma $
